Course Instructor: Bernd Neumayr, JKU

UE06: SHACL Rules

Complete the 10 tasks (1 point per task) in the 5. SHACL Rules sheet of SemAI.jar first and then transfer them to this notebook.

For each task include:

A headline including the task number

The task description

The data graph and your solution (the shapes graph including the rules) in executable form

Print out out the the data graph including derived statements (after execution of the rules).



**Preparations**

In [ ]:
# Install required packages
!pip install -q rdflib 
!pip3 install -q pyshacl    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.4.1 requires prettytable<1, but you have prettytable 2.5.0 which is incompatible.


**Imports and Functions**

In [ ]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg, sg):
    conforms, results_graph, results_text = validate(dg, shacl_graph=sg,
                                                      inference='rdfs',
                                                      abort_on_first=False,
                                                      allow_infos=False,
                                                      allow_warnings=False,
                                                      meta_shacl=False,
                                                      advanced=True,
                                                      iterate_rules=True,
                                                      inplace=True,
                                                      js=False,
                                                      debug=False)

    # Get the updated data graph
    updated_dg = dg + results_graph
    
    return conforms, updated_dg, results_text



3. TASK

The tax rate of a product category is propagated to products.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a triple rule.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<HP4>   rdf:type          <Product> ;
        :price            12 ;
        :productCategory  <Book> .

<Porsche911>  rdf:type    <Product> ;
        :price            121000 ;
        :productCategory  <Car> .

<VolvoV50>  rdf:type      <Product> ;
        :price            27000 ;
        :productCategory  <Car> .

<Car>   :taxRate  0.2 .

<Book>  :taxRate  0.1 .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
  sh:targetClass <Product> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :taxRate ;
      sh:object [sh:path (:productCategory :taxRate)]
  ].

""")


# Validate the data graph against the shapes graph
conforms, updated_dg, results_text = shacl_validate_with_rules(dg, sg)

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

# Print the updated data graph
print("\nUpdated Data Graph:")
print(updated_dg.serialize(format="turtle"))


Original Data Graph:
@prefix : <http://example.org/properties/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/entities/HP4> a <http://example.org/entities/Product>,
        rdfs:Resource ;
    :price 12 ;
    :productCategory <http://example.org/entities/Book> ;
    :taxRate 0.1 .

<http://example.org/entities/Porsche911> a <http://example.org/entities/Product>,
        rdfs:Resource ;
    :price 121000 ;
    :productCategory <http://example.org/entities/Car> ;
    :taxRate 0.2 .

<http://example.org/entities/VolvoV50> a <http://example.org/entities/Product>,
        rdfs:Resource ;
    :price 27000 ;
    :productCategory <http://example.org/entities/Car> ;
    :taxRate 0.2 .

<http://example.org/entities/Book> a rdfs:Resource ;
    :taxRate 0.1 .

:price a rdf:Property ;
    rdfs:subPropertyOf :price .

:productCategory a rdf:Property ;
    r

4. TASK

Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.

Additional requirements (not checked by the tool, you have to check them yourself):

Derive :hasFather statements using a triple rule defined as part of a node shape that has Man as target class.

Derive :hasMother statements using a triple rule defined as part of a node shape that has Woman as target class.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person>.
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<ManShape> a sh:NodeShape ;
  sh:targetClass <Man> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild] ;
      sh:predicate :hasFather ;
      sh:object sh:this;
      sh:condition [sh:class <Man>]
  ].
<WomanShape> a sh:NodeShape ;
  sh:targetClass <Woman> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild ] ;
      sh:predicate :hasMother ;
      sh:object sh:this;
      sh:condition [sh:class <Woman>]
  ].

""")

conforms, updated_dg, results_text = shacl_validate(dg,sg)   

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

print("\nUpdated Data Graph:")
print(updated_dg.serialize(format="turtle"))

Original Data Graph:
@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/entities/Diana> a <http://example.org/entities/Woman> ;
    :hasChild <http://example.org/entities/Harry>,
        <http://example.org/entities/William> .

<http://example.org/entities/Elizabeth> a <http://example.org/entities/Woman> ;
    :hasChild <http://example.org/entities/Charles> .

<http://example.org/entities/Archie> a <http://example.org/entities/Person> .

<http://example.org/entities/Charles> a <http://example.org/entities/Man> ;
    :hasChild <http://example.org/entities/Harry>,
        <http://example.org/entities/William> .

<http://example.org/entities/Harry> a <http://example.org/entities/Man> ;
    :hasChild <http://example.org/entities/Archie> .

<http://example.org/entities/William> a <http://example.org/entities/Man> .

<http://example.org/entities/Woman> rdfs:subClassOf <http://example.org/entities/Person> .

<http://exampl

5. TASK

Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.

Additional requirements (not checked by the tool, you have to check them yourself):

Derive :hasFather statements as well as :hasMother statements using triple rules defined as part of a node shape that has Person as target class.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild] ;
      sh:predicate :hasFather ;
      sh:object sh:this;
      sh:condition [sh:class <Man>]
  ], 
 [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild] ;
      sh:predicate :hasMother ;
      sh:object sh:this;
      sh:condition [sh:class <Woman>]
  ].
""")

conforms, updated_dg, results_text = shacl_validate(dg,sg)  

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

print(updated_dg.serialize(format="turtle"))

Original Data Graph:
@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/entities/Diana> a <http://example.org/entities/Woman> ;
    :hasChild <http://example.org/entities/Harry>,
        <http://example.org/entities/William> .

<http://example.org/entities/Elizabeth> a <http://example.org/entities/Woman> ;
    :hasChild <http://example.org/entities/Charles> .

<http://example.org/entities/John> a <http://example.org/entities/MalePerson> ;
    :knows <http://example.org/entities/Mary> .

<http://example.org/entities/Archie> a <http://example.org/entities/Person> .

<http://example.org/entities/Charles> a <http://example.org/entities/Man> ;
    :hasChild <http://example.org/entities/Harry>,
        <http://example.org/entities/William> .

<http://example.org/entities/Harry> a <http://example.org/entities/Man> ;
    :hasChild <http://example.org/entities/Archie> .

<http://example.org/entities/William> a <http://exampl

7. TASK

The area of a rectangle is the product of its length and its width.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
      <Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/properties/"^^xsd:anyURI ] ,
  [ sh:prefix "x"; sh:namespace "http://example.org/entities/"^^xsd:anyURI ].
   <RectangleShape> a sh:NodeShape ;
     sh:targetClass <Rectangle> ;
     sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct """
            CONSTRUCT { ?this :area ?area}
             WHERE { ?this :length ?length.
                    ?this :width ?width.
              BIND (?width * ?length AS ?area)}
      """ ;
  ].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

SyntaxError: ignored

8. TASK

The area of a rectangle is the product of its length and its width.

Additional requirements (not checked by the tool, you have to check them yourself):

Solve this using a SHACL function :multiply together with a triple rule.

In [ ]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<RectangleShape> a sh:NodeShape ;
	sh:targetClass <Rectangle> ;
        	sh:rule [
		a sh:TripleRule ;
		sh:subject sh:this ;
		sh:predicate :area ;    
		sh:object [
		:multiply ( [ sh:path :width ] [ sh:path :length ] ) ;
		] ;	   
	].

""")
 
conforms, results_graph, results_text = shacl_validate(dg,sg)  

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

Original Data Graph:
@prefix : <http://example.org/properties/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/entities/Peter> a <http://example.org/entities/Person> ;
    :length 1.72 ;
    :width 0.9 .

<http://example.org/entities/Rect1> a <http://example.org/entities/Rectangle> ;
    :length 10 ;
    :width 6 .

<http://example.org/entities/Rect2> a <http://example.org/entities/Rectangle> ;
    :length 5.5 ;
    :width 4 .



Shapes Graph:
@prefix : <http://example.org/properties/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/entities/RectangleShape> a sh:NodeShape ;
    sh:rule [ a sh:TripleRule ;
            sh:object [ :multiply ( [ sh:path :width ] [ sh:path :length ] ) ] ;
            sh:predicate :area ;
            sh:subject sh:this ] ;
    sh:targetClass <http://

10. TASK

The body mass index (:bmi) of a person is its weight divided by its height squared.

The bmi should be rounded to 2 decimal places.

Additional requirements (not checked by the tool, you have to check them yourself):

Solve this using SHACL functions (one for multiply and one for division rounded to two decimal places) and a triple rule.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Bello>  rdf:type  <Dog> ;
        :height   0.77 ;
        :weight   27 .

<Peter>  rdf:type  <Person> ;
        :height   1.78 ;
        :weight   86 .

<Jane>  rdf:type  <Person> ;
        :height   1.72 ;
        :weight   72 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
	sh:targetClass <Person> ;
        	sh:rule [
		a sh:TripleRule ;
		sh:subject sh:this ;
		sh:predicate :bmi ;    
		sh:object [
 :divide([ sh:path :weight ]
         [:multiply ( 
                      [ sh:path :height ]  
                      [ sh:path :height ]) ]
         )
		         ]; ].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

print("Original Data Graph:")
print(dg.serialize(format="turtle"))

# Print the shapes graph
print("\nShapes Graph:")
print(sg.serialize(format="turtle"))

Original Data Graph:
@prefix : <http://example.org/properties/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/entities/Bello> a <http://example.org/entities/Dog> ;
    :height 0.77 ;
    :weight 27 .

<http://example.org/entities/Jane> a <http://example.org/entities/Person> ;
    :height 1.72 ;
    :weight 72 .

<http://example.org/entities/Peter> a <http://example.org/entities/Person> ;
    :height 1.78 ;
    :weight 86 .



Shapes Graph:
@prefix : <http://example.org/properties/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/entities/PersonShape> a sh:NodeShape ;
    sh:rule [ a sh:TripleRule ;
            sh:object [ :divide ( [ sh:path :weight ] [ :multiply ( [ sh:path :height ] [ sh:path :height ] ) ] ) ] ;
            sh:predicate :bmi ;
            sh:subject sh:this ]